In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 901kB 23.5MB/s 
     |████████████████████████████████| 3.3MB 39.5MB/s 


In [ ]:
from transformers import TFMobileBertModel
import pandas as pd
from google.colab import drive
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
import json
drive.mount('/content/drive')
import numpy as np

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.60.221.130:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, -3431441669507812407),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -5155972386471334844),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8346542974526427703),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6476562038616981980),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4685444103710939326),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -386479065342659668),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, -4733120981771077749),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 174574902192142263),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.60.221.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.60.221.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
class distilBertModels:
  def __init__(self,path):
    df = pd.read_csv(path)
    self.max_seq_len = 35
    bert_name = 'google/mobilebert-uncased'
    # self.tokenizer = AlbertTokenizer.from_pretrained(bert_name,add_special_tokens=True,max_length =self.max_seq_len,pad_to_max_length = True)
    self.train_set,self.test_set = self.prepare_data(df)
    with strategy.scope():
      self.bert = TFMobileBertModel.from_pretrained(bert_name)
      self.bert.trainable = False

  def prepare_data(self,df):
    df['encoded'] = df['encoded'].apply(lambda x: json.loads(x))
    df_tr = df[df['kfold']!=0].reset_index(drop=True)
    df_ts = df[df['kfold']==0].reset_index(drop=True)
    enc_tr = list(df_tr['encoded'])
    enc_ts = list(df_ts['encoded'])
    inp_ids_tr = [np.array(k[0]) for k in enc_tr]
    att_mks_tr = [np.array(k[1]) for k in enc_tr]
    inp_ids_ts = [np.array(k[0]) for k in enc_ts]
    att_mks_ts = [np.array(k[1]) for k in enc_ts]
    x_tr_inp_ids = tf.cast(tf.squeeze(tf.stack(inp_ids_tr)),dtype=tf.int32)
    x_tr_att_mks = tf.cast(tf.squeeze(tf.stack(att_mks_tr)),dtype=tf.int32)
    x_ts_inp_ids = tf.cast(tf.squeeze(tf.stack(inp_ids_ts)),dtype=tf.int32)
    x_ts_att_mks = tf.cast(tf.squeeze(tf.stack(att_mks_ts)),dtype=tf.int32)

    encoder = LabelEncoder()
    y_tr = np_utils.to_categorical(encoder.fit_transform(df_tr.label.values))
    y_ts = np_utils.to_categorical(encoder.fit_transform(df_ts.label.values))

    x_tr = [x_tr_inp_ids,x_tr_att_mks]
    x_ts = [x_ts_inp_ids,x_ts_att_mks]

    return ((x_tr,y_tr),(x_ts,y_ts))

  # def encode_data(self,x):
  #   encoded = self.tokenizer.encode_plus(x,add_special_tokens=True,max_length =self.max_seq_len,padding = 'max_length',truncation=True,return_attention_mask=True,return_tensors='tf')
  #   return encoded
  
  def process(self,ls):
  # for name in ls:
#       his = self.compile_model(name)
#       dfs = []
#       for i in range(len(his)):
#         df = pd.DataFrame.from_dict(his[i].history)
#         df['fold'] = i+1
#         df['epoch'] = [j+1 for j in range(len(df))]
#         dfs.append(df)
#       df_fin = pd.concat(dfs,ignore_index=True)  
#       df_fin.to_csv('/content/drive/MyDrive/Bert/Folds/sent_'+name+'_folds.csv',index=False,header=True)
    for name in ls:
      his = self.compile_model(name)
      df = pd.DataFrame.from_dict(his.history)
      df['epoch'] = [i+1 for i in range(len(df))]
      df.to_csv('/content/drive/MyDrive/MOBILE_BERT/sarcasm/sarc_'+name+'.csv',index=False,header=True)
      print("\n------------------ PROCESS FOR MODEL: "+name+' COMPLETED ------------------\n')


  def compile_model(self,name):
    with strategy.scope():
      model = self.create_model(name)
      model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
    
    
    print('\n------------------ '+name+' ------------------\n')
    model.summary()
    # his = []
    # for i in range(len(self.train_set_folds)):
#       print('\n**************** FOLD '+str(i+1)+' ****************')
    his = model.fit(x = self.train_set[0],y = self.train_set[1],validation_data=(self.test_set[0],self.test_set[1]),epochs=100,callbacks=[EarlyStopping()])
    print('\n**************** TRAINING OF MODEL: '+name+' COMPLETED****************\n')
    return his
    
  def create_model(self,name):
    inputs = tf.keras.layers.Input((self.max_seq_len,),dtype = tf.int32,name="input_ids")
    mask = tf.keras.layers.Input((self.max_seq_len,),dtype = tf.int32,name="attention_mask")

    pred = self.bert([inputs,mask])[0]

    if name == 'MOBILE-BERT-CONV1D': ## 1
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(x)
      x = tf.keras.layers.GlobalAveragePooling1D()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(20,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)

    
    if name == 'MOBILE-BERT-CONV1D-BiGRU': ## 2
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
    
    if name == 'MOBILE-BERT-CONV1D-BiLSTM': ## 3
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)    
      
    if name == 'MOBILE-BERT-CONV1D-BiRNN': ## 4
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)

    if name == 'MOBILE-BERT-SEP-CONV1D': #5
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(x)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      

    if name == 'MOBILE-BERT-SEP-CONV1D-BiGRU': ## 6
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
    if name == 'MOBILE-BERT-SEP-CONV1D-BiRNN': ## 7
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
    if name == 'MOBILE-BERT-SEP-CONV1D-BiLSTM': ## 8
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
      
    if name == 'MOBILE-BERT-BiGRU': ## 9
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
    if name == 'MOBILE-BERT-BiLSTM': ## 10
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
    
    if name == 'MOBILE-BERT-BiRNN': ## 11
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
    
      
    return tf.keras.Model([inputs,mask],x)




    

  

In [ ]:
bert = distilBertModels('/content/drive/MyDrive/sarcasm data/tr_sarc_fin_mob_bert_trans_inps.csv')

Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFMobileBertModel: ['predictions___cls', 'seq_relationship___cls']
- This IS expected if you are initializing TFMobileBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMobileBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFMobileBertModel were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertModel for predictions without further training.


In [ ]:
bert.process(['MOBILE-BERT-CONV1D','MOBILE-BERT-CONV1D-BiGRU','MOBILE-BERT-CONV1D-BiLSTM','MOBILE-BERT-CONV1D-BiRNN','MOBILE-BERT-SEP-CONV1D','MOBILE-BERT-SEP-CONV1D-BiGRU'])
# bert.process(['MOBILE-BERT-CONV1D-BiRNN','MOBILE-BERT-SEP-CONV1D','MOBILE-BERT-SEP-CONV1D-BiGRU'])
# {1:'MOBILE-BERT-CONV1D',2:'MOBILE-BERT-CONV1D-BiGRU',3:'MOBILE-BERT-CONV1D-BiLSTM',4:'MOBILE-BERT-CONV1D-BiRNN',5:'MOBILE-BERT-SEP-CONV1D',6:'MOBILE-BERT-SEP-CONV1D-BiGRU',7:'MOBILE-BERT-SEP-CONV1D-BiRNN',8:'MOBILE-BERT-SEP-CONV1D-BiLSTM',9:'MOBILE-BERT-BiGRU',10:'MOBILE-BERT-BiLSTM',11:'MOBILE-BERT-BiRNN'}


------------------ MOBILE-BERT-CONV1D ------------------

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 35)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 35)]         0                                            
__________________________________________________________________________________________________
tf_mobile_bert_model (TFMobileB TFBaseModelOutputWit 24581888    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (

7349/7350 [============================>.] - ETA: 0s - loss: 1459.3131 - accuracy: 0.7621WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


7350/7350 [==============================] - 375s 46ms/step - loss: 1458.9726 - accuracy: 0.7622 - val_loss: 0.3982 - val_accuracy: 0.8209
Epoch 2/100
7350/7350 [==============================] - 310s 42ms/step - loss: 1.1270 - accuracy: 0.8197 - val_loss: 0.3711 - val_accuracy: 0.8340
Epoch 3/100
7350/7350 [==============================] - 308s 42ms/step - loss: 1.2723 - accuracy: 0.8303 - val_loss: 0.3691 - val_accuracy: 0.8349
Epoch 4/100
7350/7350 [==============================] - 314s 43ms/step - loss: 7.0164 - accuracy: 0.8347 - val_loss: 0.3653 - val_accuracy: 0.8370
Epoch 5/100
7350/7350 [==============================] - 315s 43ms/step - loss: 2.1484 - accuracy: 0.8442 - val_loss: 0.3521 - val_accuracy: 0.8430
Epoch 6/100
7350/7350 [==============================] - 314s 43ms/step - loss: 0.3436 - accuracy: 0.8480 - val_loss: 0.3484 - val_accuracy: 0.8450
Epoch 7/100
7350/7350 [==============================] - 314s 43ms/step - loss: 0.9977 - accuracy: 0.8505 - val_loss: 0.3


------------------ MOBILE-BERT-CONV1D-BiGRU ------------------

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 35)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 35)]         0                                            
__________________________________________________________________________________________________
tf_mobile_bert_model (TFMobileB TFBaseModelOutputWit 24581888    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
conv1d_5 (Conv1D)          

7350/7350 [==============================] - ETA: 0s - loss: 0.4398 - accuracy: 0.7945WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


...

7350/7350 [==============================] - ETA: 0s - loss: 52.1351 - accuracy: 0.7265WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


7350/7350 [==============================] - 411s 50ms/step - loss: 52.1291 - accuracy: 0.7265 - val_loss: 0.4026 - val_accuracy: 0.8161
Epoch 2/100
7350/7350 [==============================] - 337s 46ms/step - loss: 0.4146 - accuracy: 0.8102 - val_loss: 0.3899 - val_accuracy: 0.8229
Epoch 3/100
7350/7350 [==============================] - 332s 45ms/step - loss: 0.4625 - accuracy: 0.8166 - val_loss: 0.3863 - val_accuracy: 0.8227
Epoch 4/100
7350/7350 [==============================] - 330s 45ms/step - loss: 0.3890 - accuracy: 0.8258 - val_loss: 0.3932 - val_accuracy: 0.8193

**************** TRAINING OF MODEL: MOBILE-BERT-SEP-CONV1D COMPLETED****************


------------------ PROCESS FOR MODEL: MOBILE-BERT-SEP-CONV1D COMPLETED ------------------




------------------ MOBILE-BERT-SEP-CONV1D-BiGRU ------------------

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 35)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 35)]         0                                            
__________________________________________________________________________________________________
tf_mobile_bert_model (TFMobileB TFBaseModelOutputWit 24581888    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
separable_conv1d_2 (Sep

7350/7350 [==============================] - ETA: 0s - loss: 0.4327 - accuracy: 0.7980WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


7350/7350 [==============================] - 417s 51ms/step - loss: 0.4327 - accuracy: 0.7980 - val_loss: 0.3723 - val_accuracy: 0.8333
Epoch 2/100
7350/7350 [==============================] - 346s 47ms/step - loss: 0.3807 - accuracy: 0.8293 - val_loss: 0.3610 - val_accuracy: 0.8413
Epoch 3/100
7350/7350 [==============================] - 338s 46ms/step - loss: 0.3685 - accuracy: 0.8351 - val_loss: 0.3579 - val_accuracy: 0.8425
Epoch 4/100
7350/7350 [==============================] - 340s 46ms/step - loss: 0.3571 - accuracy: 0.8410 - val_loss: 0.3585 - val_accuracy: 0.8402

**************** TRAINING OF MODEL: MOBILE-BERT-SEP-CONV1D-BiGRU COMPLETED****************


------------------ PROCESS FOR MODEL: MOBILE-BERT-SEP-CONV1D-BiGRU COMPLETED ------------------

